In [1]:
import os

# path = "./brigadier/src/main/java/**/*.java"

from glob import glob

In [6]:
def compute_unimplemented(path):
    javafiles = list(glob(path+"/**/*.java", recursive=True))
    print(javafiles)

In [4]:
def test_unimplemented():
    expected = {"DoublePredicate"}

    result = compute_unimplemented("./src2")
    assert expected == result

In [7]:
test_unimplemented()

['./src2/example.java']


AssertionError: 

In [8]:
## Parser function
import javalang

def getAST(path: str):
    with open(path, "r") as f:
        code = f.read()
    return javalang.parse.parse(code)

In [26]:
def compute_unimplemented(path):
    javafiles = list(glob(path+"/**/*.java", recursive=True))

    gen_real_dict = {}
    for javafile in javafiles:
        myast = getAST(javafile)
        for path, node in javalang.ast.walk_tree(myast):
            if isinstance(node, javalang.tree.ClassDeclaration):
                gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                             "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                             "type": "class"}
            elif isinstance(node, javalang.tree.InterfaceDeclaration):
                gen_real_dict[node.name] = { "extends": list([sup.name for sup in node.extends]) if node.extends is not None else None,
                                             "type": "interface"}
    
    
    implemented = []
    
    # direct implementation
    for atype, props in gen_real_dict.items():
        if props["type"] == "class":
            for itype in props["implements"]:
                implemented.append(itype)
    
    implemented2 = []
    
    def add_parents(itype):
        implemented2.append(itype)
        if "extends" in gen_real_dict[itype] and gen_real_dict[itype]["extends"] is not None:
            supers = gen_real_dict[itype]["extends"]
            for sup in supers:
                add_parents(sup)
    
    for itype in set(implemented):
        add_parents(itype)
        
    unimplemented = []
    for atype, props in gen_real_dict.items():
        if props["type"] == "interface" and atype not in implemented2:
            unimplemented.append(atype)
            
    print(unimplemented)
    return set(unimplemented)

In [27]:
test_unimplemented()

['DoublePredicate']


In [48]:
def collect_class_forest(path):
    javafiles = list(glob(path+"/**/*.java", recursive=True))

    gen_real_dict = {}
    for javafile in javafiles:
        myast = getAST(javafile)
        for path, node in javalang.ast.walk_tree(myast):
            if isinstance(node, javalang.tree.ClassDeclaration):
                gen_real_dict[node.name] = { "extends": node.extends.name if node.extends is not None else None,
                                             "implements": list([atype.name for atype in node.implements]) if node.implements is not None else None,
                                             "type": "class"}
                
#     print(gen_real_dict)

    topdown = {}
    for atype, props in gen_real_dict.items():
        if props["extends"] is not None:
            sup = props["extends"]
            if sup in topdown:
                topdown[sup][atype] = {}
            else:
                topdown[sup] = {atype: {}}
        
#     print(topdown)

    forest = {}
    
    def update_children(atype):
        retdict = {}
        if atype in topdown and len(topdown[atype]) > 0:
            for child in topdown[atype]:
                retdict[child] = update_children(child)
        return retdict

    for key, value in topdown.items():
        ## Figuring out whether it is a root of a tree
        if key in gen_real_dict:
            if gen_real_dict[key]["extends"] is None or gen_real_dict[key]["extends"] not in gen_real_dict:
                forest[key] = update_children(key)
            
#     print(forest)
    return forest

In [47]:
collect_class_forest("./brigadier/src/main")

{'CommandNode': {'ArgumentCommandNode': {}, 'LiteralCommandNode': {}, 'RootCommandNode': {}}, 'Suggestion': {'IntegerSuggestion': {}}, 'ArgumentBuilder': {'RequiredArgumentBuilder': {}, 'LiteralArgumentBuilder': {}}}


{'CommandNode': {'ArgumentCommandNode': {},
  'LiteralCommandNode': {},
  'RootCommandNode': {}},
 'Suggestion': {'IntegerSuggestion': {}},
 'ArgumentBuilder': {'RequiredArgumentBuilder': {},
  'LiteralArgumentBuilder': {}}}

In [49]:
!git clone git@github.com:apache/commons-lang.git

Cloning into 'commons-lang'...
remote: Enumerating objects: 89853, done.
remote: Counting objects: 100% (12976/12976), done.
remote: Compressing objects: 100% (1175/1175), done.
remote: Total 89853 (delta 11883), reused 12177 (delta 11344), pack-reused 76877
Receiving objects: 100% (89853/89853), 26.36 MiB | 8.10 MiB/s, done.
Resolving deltas: 100% (41216/41216), done.


error: pathspec '36f98d8' did not match any file(s) known to git


In [52]:
collect_class_forest("./commons-lang/src/main")

{'CharSequenceTranslator': {'CsvEscaper': {},
  'CsvUnescaper': {},
  'NumericEntityUnescaper': {},
  'AggregateTranslator': {},
  'OctalUnescaper': {},
  'LookupTranslator': {},
  'UnicodeUnescaper': {},
  'CodePointTranslator': {'UnicodeEscaper': {'JavaUnicodeEscaper': {}},
   'UnicodeUnpairedSurrogateRemover': {},
   'NumericEntityEscaper': {}}},
 'Triple': {'MutableTriple': {}, 'ImmutableTriple': {}},
 'Pair': {'MutablePair': {}, 'ImmutablePair': {}, 'Diff': {}},
 'Strategy': {'PatternStrategy': {'CaseInsensitiveTextStrategy': {},
   'TimeZoneStrategy': {},
   'ISO8601TimeZoneStrategy': {}},
  'CopyQuotedStrategy': {},
  'NumberStrategy': {}},
 'AbstractCircuitBreaker': {'ThresholdCircuitBreaker': {},
  'EventCountCircuitBreaker': {}},
 'BackgroundInitializer': {'MultiBackgroundInitializer': {},
  'CallableBackgroundInitializer': {}},
 'StateStrategy': {'StateStrategyClosed': {}, 'StateStrategyOpen': {}},
 'StrTokenizer': {'StrBuilderTokenizer': {}},
 'StrMatcher': {'CharSetMatcher